In [ ]:
from threeML import *
from threeML import OGIPLike
import matplotlib.pyplot as plt
import numpy as np
from threeML.minimizer.minimization import FitFailed


path = '/home/tguethle/cookbook/SPI_cookbook/examples/automated_Crab/fit_Crab_374_reduced_bkg'

In [ ]:
def select_good_channels(path, max_chi2=1.1, min_chi2=0.0):
    """
    only chooes channels with a good fit. The goodnes is characterized by the chi2 value. 
    """
    chi2_list = []
    with open(f'{path}/spimodfit.log') as f:
        lines = f.readlines()
        for line in lines:
            if "Corresponding Pearson's chi2 stat / dof" in line:
                chi2 = float(line.split()[-3][:-4])
                chi2_list.append(chi2)
    chi2_list = np.array(chi2_list)
    print(chi2_list)
    max_good_channels = chi2_list < max_chi2
    min_good_channels = chi2_list > min_chi2
    good_channels = max_good_channels & min_good_channels
    channels = []
    for i, good in enumerate(good_channels):
        if good:
            channels.append(f'c{i+1}')
    return channels

def generate_channel_options(nr_channles=41):
    """
    generate options with which the fit can be performed. 
    """
    out = []
    for i in range(nr_channles):
        for j in range(i+6, nr_channles):
            out.append([f'c{i+1}-c{j+1}'])

    return out

def mahalanobis_dist(vals, cov, real_vals):
    dif = (vals - real_vals)
    return np.sqrt(np.linalg.multi_dot([dif, np.linalg.inv(cov), dif]))


In [ ]:
def run_fit(path, channels: list[str]):
    """
    run the fit with the given channels and channel option. 
    """
    s_1A = OGIPLike("sim_source",
                observation=f'{path}/spectra_sim_sourc.fits',
                response=f'{path}/spectral_response.rmf.fits')
    s_1A.set_active_measurements(*channels)

    spec = Powerlaw()

    ps = PointSource('crab',l=0,b=0,spectral_shape=spec)

    ps_model = Model(ps)

    ps_model.crab.spectrum.main.Powerlaw.piv = 100

    ps_data = DataList(s_1A)

    ps_jl = JointLikelihood(ps_model, ps_data)

    best_fit_parameters_ps, likelihood_values_ps = ps_jl.fit()

    ps_jl.restore_best_fit()

    val = np.array(best_fit_parameters_ps["value"])
    err = np.array(best_fit_parameters_ps["error"])
    cor = ps_jl.correlation_matrix
    cov = cor * err[:, np.newaxis] * err[np.newaxis, :]

    return val, cov

In [ ]:
channels = generate_channel_options(41)
real_vals = [7e-4, -2]

with open(f'{path}/fit_results.txt', 'w') as f:
    f.write('fit summary\n')
    f.write(f'total nr of combinations: {len(channels)}\n')

best_dist = 1000
for channel in channels:
    try:
        val, cov = run_fit(path, channel)
    except FitFailed:
        continue

    dist = mahalanobis_dist(val, cov, real_vals)
    with open(f'{path}/fit_results.txt', 'a') as f:
        f.write(f'channels: {channel}, values: {val}, cov: {cov}, distance: {dist}\n')
    if dist < best_dist:
        best_dist = dist
        best_channel = channel

with open(f'{path}/fit_results.txt', 'a') as f:
    f.write(f'best fit: {best_channel}, distance: {best_dist}\n')


In [ ]:

###########
# DATASET #
###########
# folder = "0374_sim"
s_1A = OGIPLike("sim_source",
                observation=f'{path}/spectra_sim_sourc.fits',
                response=f'{path}/spectral_response.rmf.fits')

###################
# ACTIVE CHANNELS #
###################
active_channels = select_good_channels(path, min_chi2=0.0)
active_energy = "50.0-200.0"
# s_1A.set_active_measurements(*active_channels)
# s_1A.set_active_measurements(active_energy)

s_1A.set_active_measurements('c11-c30')


##################
# SPECTRAL MODEL #
##################
spec = Powerlaw()

##############################
# DEFINITION OF POINT SOURCE #
##############################
ps = PointSource('crab',l=0,b=0,spectral_shape=spec)

####################
# MODEL DEFINITION #
####################
ps_model = Model(ps)

####################
# FIXED PARAMETERS #
####################
ps_model.crab.spectrum.main.Powerlaw.piv = 100

#################
# DISPLAY MODEL #
#################
ps_model.display(complete=True)



In [ ]:
###################
# DATA DEFINITION #
###################
ps_data = DataList(s_1A)

#####################
# LIKELIHOOD OBJECT #
#####################
ps_jl = JointLikelihood(ps_model, ps_data)

#######
# FIT #
#######
best_fit_parameters_ps, likelihood_values_ps = ps_jl.fit()

####################
# RESTORE BEST FIT #
####################
ps_jl.restore_best_fit()

#######################
# PLOT DATA AND MODEL #
#######################
# fig, ax = plt.subplots()
# fig = display_spectrum_model_counts(ps_jl,step=True, show_legend=False)

# from matplotlib.lines import Line2D

# legend_elements = [Line2D([0], [0], color='r', lw=2, label='1A0535+26a Model'),
#                    Line2D([0], [0], marker='+', lw=0, color='r', label='1A0535+26a')]
# fig.legend(handles=legend_elements)

# plt.savefig(f'{folder}/sim_source.pdf')

# with open(f'{folder}/fit_values', 'w') as f:
#     f.write(f"K: {best_fit_parameters_ps['value'][0]:.4} +/- {best_fit_parameters_ps['error'][0]:.3}\n")
#     f.write(f"Index: {best_fit_parameters_ps['value'][1]:.4} +/- {best_fit_parameters_ps['error'][1]:.3}\n")

In [ ]:
s_1A


In [ ]:
import numpy as np
import pickle

val = np.array(best_fit_parameters_ps["value"])
err = np.array(best_fit_parameters_ps["error"])
cor = ps_jl.correlation_matrix
cov = cor * err[:, np.newaxis] * err[np.newaxis, :]

with open(f"source_parameters.pickle", "wb") as f:
    pickle.dump((val, cov),f)

In [ ]:
import numpy as np

np.array(best_fit_parameters_ps["value"]), np.array(best_fit_parameters_ps["error"])

In [ ]:
best_fit_parameters_ps["value"][0]